# Entrega 3 - Interpretabilidad de Modelos
## Análisis SHAP y LIME para el Titanic

Este notebook contiene el análisis completo de interpretabilidad de los modelos de machine learning, incluyendo:
- Análisis SHAP (SHapley Additive exPlanations)
- Explicaciones LIME (Local Interpretable Model-agnostic Explanations)
- Visualizaciones de interpretación global y local
- Insights sobre el comportamiento del modelo